In [ ]:
import os 
import sys
m_path = os.getcwd()
sys.path.append(m_path)

import sympy as sp
import numpy as np
import scipy.io as sio
from mpi4py import MPI

from stream_functions import StreamFunctions
from boundary_conditions import BoundaryConditions
from temporal_stability import TemporalStability
from two_equation_solver import *
from claude1 import *

max_threshold = 1e6
delta_val = 0.1
s = -1

alpha_r = np.arange(0,4.01,0.25)
alpha_i = np.arange(-4,4,0.25)
alpha_R, alpha_I = np.meshgrid(alpha_r, alpha_i)
alpha = alpha_R + 1j * alpha_I

S_vals = np.logspace(-1,1,25)
L_vals = np.linspace(-2,2,25)
S_mat, L_mat = np.meshgrid(S_vals, L_vals)

sf = StreamFunctions(n_wakes=1)
bc = BoundaryConditions(sf)
D  = bc.M_non_dim
A = bc.A_non_dim
B = bc.B_non_dim
dispersion_relation = sp.simplify(sp.det(D))

for idx_SL in range(S_mat.shape[0]):
    for jdx_SL in range(S_mat.shape[1]):
        print(f'idx_SL: {idx_SL}, jdx_SL: {jdx_SL}')
        val_set = {'S': S_mat[idx_SL, jdx_SL], 'Lambda': L_mat[idx_SL, jdx_SL], 'L': 0.0, 'delta': 0.1, 's':-1}
        A_sub = A.subs(val_set)
        B_sub = B.subs(val_set)
        dispersion_relation_sub = dispersion_relation.subs(val_set)

        deriv_disp = sp.diff(dispersion_relation_sub, sf.alpha)
        print(sp.latex(deriv_disp))

        ts = TemporalStability(sf, bc, val_set)
        w_vals_imag = np.zeros_like(alpha_I)
        w_vals = np.zeros_like(alpha)

        for i in range(alpha.shape[0]):
            print(f'Row {i+1} of {alpha.shape[0]}')
            for j in range(alpha.shape[1]):
                alpha_val = alpha[i,j]
                c_val_list = ts.find_c(alpha_val)
                temp_w_vals = c_val_list * alpha_val
                temp_w_vals[np.isnan(np.abs(temp_w_vals))] = -1e10-1j*1e10
                temp_w_vals[np.isinf(np.abs(temp_w_vals))] = -1e10-1j*1e10
                temp_w_vals[np.abs(temp_w_vals) > max_threshold] = -1e10-1j*1e10
                temp_w_vals = temp_w_vals[temp_w_vals != 0 + 0j]
                if len(temp_w_vals) == 0:
                    temp_max_w_val_imag = np.array([0.0])
                else:
                    temp_max_w_val_imag = np.max(np.imag(temp_w_vals)) 
                max_w_val_idx = np.argmax(np.imag(temp_w_vals))
                w_vals[i,j] = temp_w_vals[max_w_val_idx]
                w_vals_imag[i,j] = np.imag(w_vals[i,j])

        deriv_vals = np.zeros_like(alpha)
        deriv_val_set = {'c': f'{sf.w}/{sf.alpha}'}
        deriv_disp_sub1 = deriv_disp.subs(deriv_val_set)
        for i in range(alpha.shape[0]):
            print(f'Row {i+1} of {alpha.shape[0]}')
            for j in range(alpha.shape[1]):
                deriv_disp_sub2 = deriv_disp_sub1.subs({sf.alpha: alpha[i,j], sf.w: w_vals[i,j]})
                deriv_vals[i,j] = deriv_disp_sub2.evalf()

        sio.savemat(f'C:\\Users\\tvivek\\Desktop\\multiwake_results\\w_vals_imag_Sidx{idx_SL}_Lidx{jdx_SL}.mat', 
                    {'w_vals': w_vals, 'deriv_vals':deriv_vals, 'alpha_R': alpha_R, 'alpha_I': alpha_I, 'S_mat': S_mat, 'L_mat': L_mat})

idx_SL: 0, jdx_SL: 0
1.4 \left(2.0 \left(- 0.11 \alpha c - 0.11 \alpha - 4.0\right) e^{- 1.25 \alpha} - 2.0 \left(- 0.09 \alpha c - 0.09 \alpha - 4.0\right) e^{- 0.35 \alpha} - \left(0.009 \alpha^{2} c^{2} - 0.018 \alpha^{2} c - 0.027 \alpha^{2} - 0.22 \alpha c + 1.38 \alpha - 8.0\right) e^{- 1.05 \alpha} + \left(0.011 \alpha^{2} c^{2} - 0.022 \alpha^{2} c - 0.033 \alpha^{2} - 0.18 \alpha c + 1.42 \alpha - 8.0\right) e^{- 0.15 \alpha}\right) e^{0.7 \alpha} + 2 \left(2.0 \left(- 0.11 c - 0.11\right) e^{- 1.25 \alpha} - 2.0 \left(- 0.09 c - 0.09\right) e^{- 0.35 \alpha} - 2.5 \left(- 0.11 \alpha c - 0.11 \alpha - 4.0\right) e^{- 1.25 \alpha} + 0.7 \left(- 0.09 \alpha c - 0.09 \alpha - 4.0\right) e^{- 0.35 \alpha} - \left(0.018 \alpha c^{2} - 0.036 \alpha c - 0.054 \alpha - 0.22 c + 1.38\right) e^{- 1.05 \alpha} + \left(0.022 \alpha c^{2} - 0.044 \alpha c - 0.066 \alpha - 0.18 c + 1.42\right) e^{- 0.15 \alpha} + 1.05 \left(0.009 \alpha^{2} c^{2} - 0.018 \alpha^{2} c - 0.027 \alpha^{2} - 0

C:\Users\tvivek\AppData\Local\Temp\ipykernel_33504\351038111.py:56: RuntimeWarning: invalid value encountered in multiply
  temp_w_vals = c_val_list * alpha_val


Row 2 of 32
Row 3 of 32
Row 4 of 32
Row 5 of 32
Row 6 of 32
Row 7 of 32
Row 8 of 32
Row 9 of 32
Row 10 of 32
Row 11 of 32
Row 12 of 32
Row 13 of 32
Row 14 of 32
Row 15 of 32
Row 16 of 32
Row 17 of 32
Row 18 of 32
Row 19 of 32
Row 20 of 32
Row 21 of 32
Row 22 of 32
Row 23 of 32
Row 24 of 32
Row 25 of 32
Row 26 of 32
Row 27 of 32
Row 28 of 32
Row 29 of 32
Row 30 of 32
Row 31 of 32
Row 32 of 32
Row 1 of 32
Row 2 of 32
Row 3 of 32
Row 4 of 32
Row 5 of 32
Row 6 of 32
Row 7 of 32
Row 8 of 32
Row 9 of 32
Row 10 of 32
Row 11 of 32
Row 12 of 32
Row 13 of 32
Row 14 of 32
Row 15 of 32
Row 16 of 32
Row 17 of 32
Row 18 of 32
Row 19 of 32
Row 20 of 32
Row 21 of 32
Row 22 of 32
Row 23 of 32
Row 24 of 32
Row 25 of 32
Row 26 of 32
Row 27 of 32
Row 28 of 32
Row 29 of 32
Row 30 of 32
Row 31 of 32
Row 32 of 32
idx_SL: 0, jdx_SL: 1
1.4 \left(2.0 \left(- 0.112115276586286 \alpha c - 0.112115276586286 \alpha - 4.0\right) e^{- 1.25 \alpha} - 2.0 \left(- 0.0878847234137141 \alpha c - 0.0878847234137141 \alpha 

KeyboardInterrupt: 